## 개선된 CBOW모델 구현
- input -> W로 전달되는 부분의 효율적인 학습을 구현한 **Embedding**층을 적용하고
- h -> out(loss)로 전달되는 부분의 효과적인 학습을 구현한 **Negative Sampling**계층을 적용

In [27]:
import numpy as np
import sys
from mh_common.mh_layers import Embedding
sys.path.append(r'C:\Users\kang_lp\OneDrive - UOS\bitamin\dl_nlp_study\deep-learning-from-scratch-2-master')

from ch04.negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        #가중치 초기화
        W_in = 0.01 * np.random.randn(V,H).astype('f')
        W_out = 0.01 * np.random.randn(V,H).astype('f')
        
        #계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power = 0.75, sample_size = 5)
        
        #모든 가중치와 기울기를 []안에 모으는 작업
        layers = self.in_layers + [self.ns_loss] #기존 in_layer리스트 안에 self.ns_loss의 객체들을 추가하는 것
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
        
        #단어의 분산 표현을 저장
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        #contexts는 target을 둘러싸는 window단어들의 단어ID를 담은 행렬
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout = 1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

In [28]:
#CBOW모델 훈련
import pickle
from common import config
from mh_common.mh_trainer import Trainer
from mh_common.mh_optimizer import Adam
from mh_common.mh_utils import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

#하이퍼 파라미터 설정
window_size = 5 #윈도우 사이즈가 5, 따라서 입력층이 10(5 * 2)개로 존재하게 됨
hidden_size = 100 #100개의 뉴런을 가지는 은닉층
batch_size = 100
max_epoch = 10 #총 훈련횟수

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

#모델 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)

#optimizer 생성
optimizer = Adam()
trainer = Trainer(model, optimizer)

#학습시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

word_vecs = model.word_vecs #분산표현을 모델에 저장
if config.GPU:
    word_vecs = to_cpu(word_vecs)

#학습된 가중치들과 word_to_id, id_to_word를 pkl파일로 저장
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_param.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

    #노트북으로는 학습시간이 너무 길어서 작업을 중단함

| epoch 1 | mini-batch 1 / 9295 | 시간 0[s] |avg_loss 4.16
| epoch 1 | mini-batch 21 / 9295 | 시간 2[s] |avg_loss 4.16
| epoch 1 | mini-batch 41 / 9295 | 시간 3[s] |avg_loss 4.15
| epoch 1 | mini-batch 61 / 9295 | 시간 5[s] |avg_loss 4.13
| epoch 1 | mini-batch 81 / 9295 | 시간 7[s] |avg_loss 4.05
| epoch 1 | mini-batch 101 / 9295 | 시간 8[s] |avg_loss 3.93
| epoch 1 | mini-batch 121 / 9295 | 시간 10[s] |avg_loss 3.79
| epoch 1 | mini-batch 141 / 9295 | 시간 12[s] |avg_loss 3.63
| epoch 1 | mini-batch 161 / 9295 | 시간 14[s] |avg_loss 3.50
| epoch 1 | mini-batch 181 / 9295 | 시간 15[s] |avg_loss 3.37
| epoch 1 | mini-batch 201 / 9295 | 시간 17[s] |avg_loss 3.24
| epoch 1 | mini-batch 221 / 9295 | 시간 19[s] |avg_loss 3.17
| epoch 1 | mini-batch 241 / 9295 | 시간 20[s] |avg_loss 3.10
| epoch 1 | mini-batch 261 / 9295 | 시간 22[s] |avg_loss 3.02
| epoch 1 | mini-batch 281 / 9295 | 시간 24[s] |avg_loss 2.98
| epoch 1 | mini-batch 301 / 9295 | 시간 25[s] |avg_loss 2.92
| epoch 1 | mini-batch 321 / 9295 | 시간 27[s] |avg_lo

| epoch 1 | mini-batch 2701 / 9295 | 시간 234[s] |avg_loss 2.47
| epoch 1 | mini-batch 2721 / 9295 | 시간 236[s] |avg_loss 2.45
| epoch 1 | mini-batch 2741 / 9295 | 시간 238[s] |avg_loss 2.47
| epoch 1 | mini-batch 2761 / 9295 | 시간 239[s] |avg_loss 2.47
| epoch 1 | mini-batch 2781 / 9295 | 시간 241[s] |avg_loss 2.48
| epoch 1 | mini-batch 2801 / 9295 | 시간 243[s] |avg_loss 2.43


KeyboardInterrupt: 

In [3]:
#ptb 데이터 형태 살펴보기
import sys
sys.path.append(r'C:\Users\kang_lp\OneDrive - UOS\bitamin\dl_nlp_study\deep-learning-from-scratch-2-master')

from dataset import ptb
corpus, word_to_id, id_to_word = ptb.load_data('train')

In [8]:
type(corpus)

numpy.ndarray

In [7]:
corpus #문서의 내용들이 통으로 하나의 문장으로 존재

array([ 0,  1,  2, ..., 39, 26, 24])

In [5]:
len(corpus)

929589